# Проект SQL

**Цель исследования** 
Проанализировать базу данных купленного сервиса по чтению книг по подписке с целью свформулировать ценностное предложение для нового продукта.

**Задачи**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


## Посчитайте, сколько книг вышло после 1 января 2000 года

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
''' 
# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 книг вышло после 1 января 2000 года

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [3]:
query1 = '''
SELECT b.book_id,
       b.title ,
       re.count,
       ra.avg
FROM books AS b
LEFT JOIN (
           SELECT COUNT(review_id) AS count, 
                  book_id
           FROM reviews 
           GROUP BY book_id) AS re ON re.book_id = b.book_id
LEFT JOIN (
           SELECT AVG(rating) AS avg,
                  book_id
           FROM ratings 
           GROUP BY book_id
           ORDER BY book_id) AS ra ON ra.book_id = b.book_id           

''' 
# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query1, con = engine)

,book_id,title,count,avg
0,1,'Salem's Lot,2.0,3.666667
1,2,1 000 Places to See Before You Die,1.0,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
3,4,1491: New Revelations of the Americas Before C...,2.0,4.500000
4,5,1776,4.0,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.666667
996,997,Xenocide (Ender's Saga #3),3.0,3.400000
997,998,Year of Wonders,4.0,3.200000
998,999,You Suck (A Love Story #2),2.0,4.500000


Были посчитаны количество обзоров и средняя оценка для каждой книги

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [4]:
query2 = '''
SELECT publisher,
       publisher_id
FROM publishers
WHERE publisher_id IN (SELECT publisher_id
FROM books
WHERE num_pages > 50
GROUP BY publisher_id 
ORDER BY COUNT(book_id) DESC
LIMIT 1)
''' 
# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query2, con = engine)

,publisher,publisher_id
0,Penguin Books,212


Издательство выпустившее наибольшее количество книг - Penguin Books

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query3 = '''
SELECT fin.avtor,
       AVG(fin.rat)
FROM 
(SELECT a.author AS avtor,
       COUNT(b.book_id) AS col,
       AVG(r.rating) AS rat
FROM ratings AS r
LEFT JOIN books AS b ON b.book_id=r.book_id
LEFT JOIN authors AS a ON a.author_id = b.author_id
GROUP BY a.author, b.book_id
HAVING COUNT(r.rating_id)>=50
ORDER BY rat DESC) AS fin
GROUP BY fin.avtor
ORDER BY AVG(fin.rat) DESC
LIMIT 1
''' 
# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query3, con = engine)

,avtor,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré с средним рейтингом 4,28

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [6]:
query4 = '''
SELECT
    AVG(a.count)
FROM
    (
    SELECT username,
           COUNT(review_id) AS count
    FROM reviews
    WHERE username IN (
                   SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT(rating_id) > 50
                   )
    GROUP BY username) AS a


''' 
# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query4, con = engine)

,avg
0,24.333333


Среднее количество обзоров, оставленных активными пользователями (оставившие более 50 оценок) состовляет чуть больше 24.

## Выводы

В целом по данным можно сделать следующие выводы:

- 819 книг вышло после 1 января 2000 года
- Были посчитаны количество обзоров и средняя оценка для каждой книги
- Издательство выпустившее наибольшее количество книг - Penguin Books
- Автор с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré с средним рейтингом 4,28
- Среднее количество обзоров, оставленных активными пользователями (оставившие более 50 оценок) состовляет чуть больше 24